In [2]:
def bellman_ford_ordered(graph, source, edge_order):
    """
    Implements the Bellman-Ford algorithm following a specific edge relaxation order.

    Args:
        graph: A dictionary where keys are nodes and values are dictionaries of neighbors and edge weights
        source: The source vertex
        edge_order: List of (u,v) tuples specifying the order of edge relaxations

    Returns:
        distances: Dictionary of distances from source to each vertex
        predecessors: Dictionary of predecessors for each vertex
        iterations: List of (distances, predecessors) pairs after each pass
    """
    # Initialize distances and predecessors
    distances = {vertex: float('infinity') for vertex in graph}
    distances[source] = 0
    predecessors = {vertex: None for vertex in graph}

    # Store the state after each pass
    iterations = []

    # Number of vertices in the graph
    V = len(graph)

    # Perform |V| - 1 passes
    for i in range(V - 1):
        # Make a deep copy of current state
        current_distances = distances.copy()
        current_predecessors = predecessors.copy()

        # Relax edges in the specified order
        for u, v in edge_order:
            if u in graph and v in graph[u]:
                weight = graph[u][v]
                if distances[u] != float('infinity') and distances[u] + weight < distances[v]:
                    distances[v] = distances[u] + weight
                    predecessors[v] = u

        # Store state after this pass
        iterations.append((current_distances.copy(), current_predecessors.copy()))

    # Check for negative-weight cycles
    has_negative_cycle = any(
        distances[u] != float('infinity') and distances[u] + graph[u][v] < distances[v]
        for u in graph
        for v in graph[u]
    )

    return distances, predecessors, iterations, has_negative_cycle

# Create the graph from the image
graph = {
    's': {'t': 6, 'y': 7},
    't': {'x': 5, 'y': -4, 'z': -2},
    'y': {'z': 9, 't': 8},
    'x': {'t': -2},
    'z': {'x': 7, 's': 2}
}

# Define the edge order as shown in the image
edge_order = [
    ('t', 'x'), ('t', 'y'), ('t', 'z'),
    ('x', 't'),
    ('y', 'x'), ('y', 'z'),
    ('z', 'x'), ('z', 's'),
    ('s', 't'), ('s', 'y')
]

# Run the algorithm
source = 's'
distances, predecessors, iterations, has_negative_cycle = bellman_ford_ordered(graph, source, edge_order)

# Print results after each pass
print(f"Execution of Bellman-Ford algorithm on the example graph\n")
print(f"Source vertex: {source}")

# Map the state to what's shown in the diagrams
print("\nInitial state (diagram a):")
print("Vertex | d | π")
print("------|---|---")
initial_d = {'s': 0, 't': float('infinity'), 'x': float('infinity'), 'y': float('infinity'), 'z': float('infinity')}
initial_pi = {'s': None, 't': None, 'x': None, 'y': None, 'z': None}
for vertex in ['s', 't', 'x', 'y', 'z']:
    d_str = "∞" if initial_d[vertex] == float('infinity') else str(initial_d[vertex])
    pred = initial_pi[vertex]
    print(f"  {vertex}   | {d_str} | {pred if pred else '-'}")

# Show the states after each relaxation pass
for i, (iter_distances, iter_predecessors) in enumerate(iterations):
    print(f"\nAfter pass {i+1} (diagram {chr(98+i)}):")
    print("Vertex | d | π")
    print("------|---|---")
    for vertex in ['s', 't', 'x', 'y', 'z']:
        d_str = "∞" if iter_distances[vertex] == float('infinity') else str(iter_distances[vertex])
        pred = iter_predecessors[vertex]
        print(f"  {vertex}   | {d_str} | {pred if pred else '-'}")

print("\nFinal state (diagram e):")
print("Vertex | d | π")
print("------|---|---")
for vertex in ['s', 't', 'x', 'y', 'z']:
    d_str = "∞" if distances[vertex] == float('infinity') else str(distances[vertex])
    pred = predecessors[vertex]
    print(f"  {vertex}   | {d_str} | {pred if pred else '-'}")

print(f"\nNegative cycle detected: {has_negative_cycle}")

# Verify with expected results from image
expected_d = {'s': 0, 't': 2, 'x': 4, 'y': 7, 'z': 2}
expected_pi = {'s': None, 't': 'y', 'x': 'z', 'y': 's', 'z': 't'}

print("\nComparison with expected final values from the image:")
match = True
for vertex in graph:
    if distances[vertex] != expected_d.get(vertex) or predecessors[vertex] != expected_pi.get(vertex):
        match = False
        print(f"Mismatch for vertex {vertex}:")
        print(f"  Expected: d={expected_d.get(vertex)}, π={expected_pi.get(vertex)}")
        print(f"  Got:      d={distances[vertex]}, π={predecessors[vertex]}")

if match:
    print("Our algorithm results match the final state shown in the image!")
else:
    print("There are differences between our results and what's shown in the image.")

Execution of Bellman-Ford algorithm on the example graph

Source vertex: s

Initial state (diagram a):
Vertex | d | π
------|---|---
  s   | 0 | -
  t   | ∞ | -
  x   | ∞ | -
  y   | ∞ | -
  z   | ∞ | -

After pass 1 (diagram b):
Vertex | d | π
------|---|---
  s   | 0 | -
  t   | ∞ | -
  x   | ∞ | -
  y   | ∞ | -
  z   | ∞ | -

After pass 2 (diagram c):
Vertex | d | π
------|---|---
  s   | 0 | -
  t   | 6 | s
  x   | ∞ | -
  y   | 7 | s
  z   | ∞ | -

After pass 3 (diagram d):
Vertex | d | π
------|---|---
  s   | 0 | -
  t   | 6 | s
  x   | 11 | t
  y   | 2 | t
  z   | 4 | t

After pass 4 (diagram e):
Vertex | d | π
------|---|---
  s   | 0 | -
  t   | 6 | s
  x   | 11 | t
  y   | 2 | t
  z   | 4 | t

Final state (diagram e):
Vertex | d | π
------|---|---
  s   | 0 | -
  t   | 6 | s
  x   | 11 | t
  y   | 2 | t
  z   | 4 | t

Negative cycle detected: False

Comparison with expected final values from the image:
Mismatch for vertex t:
  Expected: d=2, π=y
  Got:      d=6, π=s
Mismatch